# 地理空间分析工作计划：缓冲区与网格对齐

这是一个标准的地理空间分析工作流，旨在实现坐标提取、缓冲区生成、网格对齐及最终坐标导出的任务。该计划分为5个主要阶段。


## 🛠️ 阶段一：数据准备与读取 (Data Preparation & Ingestion)

**目标**：将原始文件中的坐标转换为程序可识别的地理空间对象。

1.  **确定输入格式**：
    * 明确输入文件格式（如 `.csv`, `.xlsx`, `.shp`, `.geojson`）。
    * 明确坐标系（CRS），例如：WGS84 (EPSG:4326)。
2.  **读取坐标数据**：
    * 使用 `pandas`（表格）或 `geopandas`（矢量）读取数据。
    * 清洗数据（处理缺失值，确保坐标为数值类型）。
3.  **转换为地理数据框 (GeoDataFrame)**：
    * 利用 `geopandas.GeoDataFrame` 将原始坐标转换为几何点 (Point Geometry)。
    * **关键动作**：为数据指定正确的坐标系。


In [11]:
import pandas as pd
from glob import glob
from tqdm import tqdm
all_dhs = ("/nas/houce/global_slum/DHS_data/calculated_poverty_score/*.csv")
for i, path in tqdm(enumerate(glob(all_dhs)), total=len(glob(all_dhs))):
    if i == 0:
        all_dhs_df = pd.read_csv(path)
    else:
        dhs_df = pd.read_csv(path)
        all_dhs_df = pd.concat([all_dhs_df, dhs_df], axis=0)

100%|██████████| 112/112 [01:02<00:00,  1.80it/s]


In [12]:
all_dhs_df

,hv000,hv001,hv002,hv003,hv007,hv006,hv016,hv213,hv214,hv215,...,total_score,hv230b,hv232b,hv232c,hv232d,hv232e,hv232y,hv201a.1,hv230a.1,hv232.1
0,DR5,1226,1451,1.0,2007,6,18,33.0,31.0,31.0,...,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DR5,1226,2311,1.0,2007,6,18,33.0,31.0,31.0,...,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DR5,1226,1241,1.0,2007,6,20,33.0,31.0,31.0,...,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DR5,1226,2211,2.0,2007,6,17,33.0,31.0,31.0,...,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DR5,1226,2581,1.0,2007,6,18,33.0,31.0,31.0,...,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11078,ZA7,750,80,3.0,2016,8,29,34.0,33.0,31.0,...,7.0,1.0,0.0,NaN,NaN,NaN,1.0,0.0,0.0,1.0
11079,ZA7,750,81,2.0,2016,8,29,34.0,33.0,31.0,...,7.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN
11080,ZA7,750,82,1.0,2016,8,29,33.0,33.0,31.0,...,9.0,1.0,0.0,NaN,NaN,NaN,1.0,1.0,0.0,1.0
11081,ZA7,750,84,1.0,2016,8,29,34.0,33.0,31.0,...,6.0,1.0,0.0,NaN,NaN,NaN,0.0,1.0,0.0,0.0


In [13]:
all_coords = all_dhs_df[['hv007', 'URBAN_RURA', 'LATNUM', 'LONGNUM']]
all_coords.drop_duplicates(inplace=True)
all_coords.reset_index(drop=True, inplace=True)

/tmp/ipykernel_3251/421142926.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_coords.drop_duplicates(inplace=True)



## 📐 阶段二：缓冲区分析 (Buffer Generation)

**目标**：围绕每个坐标点创建具有实际物理意义（如米）的覆盖范围。

1.  **投影转换 (Projecting)**：
    * **核心痛点**：基于经纬度的缓冲区计算不准确（单位是度）。
    * **动作**：必须将数据转换到以“米”为单位的投影坐标系（如 UTM 或 EPSG:3857）。
2.  **执行缓冲 (Buffering)**：
    * 设定缓冲区半径（例如：500米）。
    * 对每个点生成圆形的 Polygon（多边形）。
3.  **(可选) 合并缓冲区**：
    * 决定是否将重叠的缓冲区融合（Dissolve）为一个大的不规则多边形。



In [14]:
import geopandas as gpd
from shapely.geometry import Point

# 创建GeoDataFrame
gdf = gpd.GeoDataFrame(
    all_coords,
    geometry=[Point(lon, lat) for lat, lon in zip(all_coords['LATNUM'], all_coords['LONGNUM'])],
    crs="EPSG:4326"
).reset_index(drop=True)

# 投影到米制坐标系（Web Mercator）
gdf = gdf.to_crs(epsg=3857)

# 设置缓冲区半径（米）
gdf['buffer_radius'] = gdf['URBAN_RURA'].map({'U': 2000, 'R': 5000})

# 生成缓冲区
gdf['buffer'] = gdf.geometry.buffer(gdf['buffer_radius'])
gdf['buffer'] = gdf['buffer'].to_crs(epsg=4326)

# 结果gdf包含缓冲区多边形
gdf.head()

,hv007,URBAN_RURA,LATNUM,LONGNUM,geometry,buffer_radius,buffer
0,2007,U,18.435688,-69.985077,POINT (-7790703.135 2088608.655),2000,"POLYGON ((-69.96711 18.43569, -69.9672 18.4340..."
1,2007,U,18.443165,-69.970033,POINT (-7789028.444 2089486.037),2000,"POLYGON ((-69.95207 18.44316, -69.95215 18.441..."
2,2007,U,18.480197,-69.947467,POINT (-7786516.409 2093832.087),2000,"POLYGON ((-69.9295 18.4802, -69.92959 18.47853..."
3,2007,U,18.451807,-69.969068,POINT (-7788921.021 2090500.172),2000,"POLYGON ((-69.9511 18.45181, -69.95119 18.4501..."
4,2007,U,18.425700,-69.971196,POINT (-7789157.909 2087436.682),2000,"POLYGON ((-69.95323 18.4257, -69.95332 18.4240..."


## ▦ 阶段三：网格构建与对齐 (Grid Generation & Alignment)

**目标**：创建参考网格，并准备好与缓冲区进行交互。

1.  **定义网格规范**：
    * 确定网格的范围（Extent）：通常覆盖所有缓冲区的最大边界框 (Total Bounds)。
    * 确定网格大小（Resolution）：例如 100m x 100m 的方格。
2.  **生成网格**：
    * 利用代码在指定范围内生成规则的矩形多边形网格。
    * 将网格转换为一个 `GeoDataFrame`。
3.  **统一坐标系**：
    * 确保网格的坐标系与缓冲区的坐标系**完全一致**。



## 🎯 阶段四：空间叠加与标注 (Spatial Overlay & Labeling)

**目标**：找出哪些网格“击中”了缓冲区，并进行标记。

1.  **空间连接 (Spatial Join)**：
    * 使用 `geopandas.sjoin()` 函数。这是核心步骤。
    * *筛选动作*：保留那些与缓冲区有交集 (`intersects`) 的网格单元。
2.  **标注位置**：
    * 为被选中的网格单元添加标记（例如，增加一列 `is_hit = True`）。
    * 为每个网格分配唯一 ID，以便后续追踪。



In [15]:
import numpy as np
import pandas as pd

# 全球范围
lon_min, lon_max = -180, 180
lat_min, lat_max = -90, 90
grid_size = 0.25

# 生成网格边界
lon_edges = np.arange(lon_min, lon_max, grid_size)
lat_edges = np.arange(lat_min, lat_max, grid_size)

# 构建网格DataFrame
grid_list = []
for lon in lon_edges:
    for lat in lat_edges:
        grid_list.append({
            'lon_min': lon,
            'lat_min': lat,
            'lon_max': lon + grid_size,
            'lat_max': lat + grid_size
        })

grid_df = pd.DataFrame(grid_list)
grid_df.head()

,lon_min,lat_min,lon_max,lat_max
0,-180.0,-90.00,-179.75,-89.75
1,-180.0,-89.75,-179.75,-89.50
2,-180.0,-89.50,-179.75,-89.25
3,-180.0,-89.25,-179.75,-89.00
4,-180.0,-89.00,-179.75,-88.75


In [16]:
from shapely.geometry import box

# 将grid_df转换为GeoDataFrame
grid_gdf = gpd.GeoDataFrame(
    grid_df,
    geometry=[
        box(row['lon_min'], row['lat_min'], row['lon_max'], row['lat_max'])
        for _, row in grid_df.iterrows()
    ],
    crs="EPSG:4326"
)

# 只保留缓冲区多边形
buffer_gdf = gdf[['hv007', 'URBAN_RURA', 'LATNUM', 'LONGNUM', 'buffer']].copy()
buffer_gdf = buffer_gdf.rename(columns={'buffer': 'geometry'})
buffer_gdf = buffer_gdf.set_geometry('geometry')
buffer_gdf.crs = "EPSG:4326"

# 空间连接（intersects）
hit_grid = gpd.sjoin(grid_gdf, buffer_gdf, how='inner', predicate='intersects')
hit_grid_counts = hit_grid.groupby(['LATNUM','LONGNUM']).size().reset_index(name='hit_count')
hit_grid = hit_grid.merge(hit_grid_counts, on=['LATNUM','LONGNUM'], how='left')
hit_grid = gpd.GeoDataFrame(
    hit_grid,
    geometry=hit_grid.geometry,
    crs="EPSG:4326"
)
hit_grid.head()
# hit_grid.to_file("/nas/houce/global_slum/DHS_data/DHS_coords_in_grid.geojson", driver='GeoJSON')
hit_grid_unique = hit_grid.drop_duplicates(subset=['geometry', 'hv007']).reset_index(drop=True)
# hit_grid_unique.to_file("/nas/houce/global_slum/DHS_data/DHS_coords_in_grid_unique.geojson", driver='GeoJSON')


## 和0.25度网格对齐,生成下载的列表

In [17]:
from shapely.geometry import box

def _to_geometry_unit(lon_min, lat_min, lon_max, lat_max):
    return box(lon_min, lat_min, lon_max, lat_max)

exist_patches = gpd.read_file("/nas/houce/Alphaearth_embedding/metadata/all_grid_cells_5x5_merged.geojson")
exist_patches = exist_patches[exist_patches['file_path_copied'].notna()].reset_index(drop=True)
exist_patches['geometry_unit'] = exist_patches.apply(
    lambda row: _to_geometry_unit(row['lon_min'], row['lat_min'], row['lon_max'], row['lat_max']),
    axis=1
)

exist_patches = exist_patches.rename(columns={'geometry': 'geometry_old',
                          'geometry_unit': 'geometry'})

In [18]:
# 对齐 hit_grid_unique 和 exist_patches 的网格（经纬度和 geometry）
# 只保留两者都存在的网格（交集）

# 先准备需要对齐的字段
merge_cols = ['lon_min', 'lon_max', 'lat_min', 'lat_max']

# 进行内连接（intersection），只保留两者都存在的网格
aligned_grid = hit_grid_unique.merge(
    exist_patches[merge_cols + ['geometry']],
    on=merge_cols,
    suffixes=('_hit', '_exist'),
    how='left'
)
aligned_grid = aligned_grid.drop_duplicates(subset=['geometry_hit', 'hv007']).reset_index(drop=True)
aligned_grid.head()

,lon_min,lat_min,lon_max,lat_max,geometry_hit,index_right,hv007,URBAN_RURA,LATNUM,LONGNUM,hit_count,geometry_exist
0,-82.00,12.25,-81.75,12.50,"POLYGON ((-81.75 12.25, -81.75 12.5, -82 12.5,...",31914,2010,R,12.508607,-81.718837,4,None
1,-82.00,12.50,-81.75,12.75,"POLYGON ((-81.75 12.5, -81.75 12.75, -82 12.75...",31914,2010,R,12.508607,-81.718837,4,None
2,-81.75,12.25,-81.50,12.50,"POLYGON ((-81.5 12.25, -81.5 12.5, -81.75 12.5...",31914,2010,R,12.508607,-81.718837,4,None
3,-81.75,12.50,-81.50,12.75,"POLYGON ((-81.5 12.5, -81.5 12.75, -81.75 12.7...",31914,2010,R,12.508607,-81.718837,4,None
4,-81.50,-4.75,-81.25,-4.50,"POLYGON ((-81.25 -4.75, -81.25 -4.5, -81.5 -4....",16528,2009,U,-4.658378,-81.308259,1,None


In [19]:
import json
import os
save_name = f"All_exist_slum"
os.makedirs(f'/nas/houce/Alphaearth_embedding/GEE_extracted/{save_name}/metadata', exist_ok=True)
if not os.path.exists(f'/code/GEE_download/{save_name}/{save_name}_grid_cells.json'):
    grid_dict = {}
    for i, row in aligned_grid[aligned_grid['geometry_exist'].isna()].iterrows():
        lon_min = row['lon_min']
        lat_min = row['lat_min']
        lon_max = row['lon_max']
        lat_max = row['lat_max']
        time = row['hv007']
        cell_name = f"{save_name}_grid_{i}"
        grid_dict[cell_name] = {
            "lon_min": f"{float(lon_min):.4f}",
            "lon_max": f"{float(lon_max):.4f}",
            "lat_min": f"{float(lat_min):.4f}",
            "lat_max": f"{float(lat_max):.4f}",
            "time_start": f"{time}-01-01",
            "time_end": f"{time}-12-31"
        }

    with open(f'/nas/houce/Alphaearth_embedding/GEE_extracted/{save_name}/metadata/{save_name}.json', 'w') as f:
        json.dump(grid_dict, f, indent=4)

    print(f"Created {len(grid_dict)} grid cells and saved to {save_name}.json")

else:
    grid_dict = json.load(open(f'/nas/houce/Alphaearth_embedding/GEE_extracted/{save_name}/metadata/{save_name}.json', 'r'))
pd.DataFrame(grid_dict).T.reset_index().rename(columns={'index':'grid_name'}).to_csv(f'/nas/houce/Alphaearth_embedding/GEE_extracted/{save_name}/metadata/{save_name}_grid_cells.csv', index=False)

Created 21944 grid cells and saved to All_exist_slum.json


In [20]:
def generate_code_script(batch_size, save_name, grid_dict, mode='python'):
    regions = []
    for cell_name, coords in grid_dict.items():
        region = {
            'name': cell_name,
            'geometry': f"ee.Geometry.Rectangle([{coords['lon_min']}, {coords['lat_min']}, {coords['lon_max']}, {coords['lat_max']}])",
            'time_start': coords['time_start'],
            'time_end': coords['time_end']
        }
        regions.append(region)

    if mode == "GEE":
        gee_code = """// =================================================================================
        // **第二步：设置通用的参数**
        // =================================================================================

        // 加载影像集合
        var dataset = ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL');

        // 开始和结束时间
        var date_s = '2023-01-01';
        var date_e = '2024-01-01';

        // 下载分辨率
        var resolution = 10;

        // 可视化参数
        var visParams = {min: -0.3, max: 0.3, bands: ['A01', 'A16', 'A09']};


        // =================================================================================
        // **第三步：循环遍历每个区域并创建导出任务**
        // =================================================================================

        // 使用 forEach 循环来处理 'regions' 列表中的每一个元素
        regions.forEach(function(region) {

            // 1. 筛选影像
            // 注意：这里的 filterBounds 使用的是当前循环中的 region.geometry
            var image = dataset
            .filterDate(date_s, date_e)
            .filterBounds(region.geometry)
            .first();

            // 2. 在地图上显示影像（可选，方便检查）
            // 为了区分，我们用区域的名字来命名图层
            Map.addLayer(image, visParams, region.name + ' embeddings');
            Map.centerObject(region.geometry, 10); // 将地图中心移动到当前区域

            // 3. 为当前区域创建导出任务
            Export.image.toDrive({
            image: image,
            // 在描述和文件名中加入区域名称，以保证每个任务和文件都独一无二
            description: 'Satellite_Embedding_' + region.name,
            folder: 'GEE_Exports', // Google Drive 中的文件夹
            fileNamePrefix: 'Satellite_Embedding_' + region.name, // 文件名前缀
            scale: resolution,
            region: region.geometry, // 使用当前循环中的区域范围
            maxPixels: 1e13
            });
        });"""


        # Format the output in JavaScript syntax for Earth Engine
        js_regions = "var regions = [\n"
        for region in regions:
            js_regions += f"  {{\n"
            js_regions += f"    name: '{region['name']}',\n"
            js_regions += f"    geometry: {region['geometry']}\n"
            js_regions += f"    time_start: {region['time_start']}\n"
            js_regions += f"    time_end: {region['time_end']}\n"
            js_regions += f"  }},\n"
        js_regions += "];"

        
        batched_regions = []
        for i in range(0, len(regions), batch_size):
            batch = regions[i:i+batch_size]
            batched_regions.append(batch)

        # Create separate JavaScript files for each batch
        for batch_idx, batch in enumerate(batched_regions):
            batch_js = "var regions = [\n"
            for region in batch:
                batch_js += f"  {{\n"
                batch_js += f"    name: '{region['name']}',\n"
                batch_js += f"    geometry: {region['geometry']}\n"
                batch_js += f"    time_start: {region['time_start']}\n"
                batch_js += f"    time_end: {region['time_end']}\n"
                batch_js += f"  }},\n"
            batch_js += "];" + "\n" + "\n" + gee_code

            # Create the directory if it doesn't exist
            os.makedirs(f'/nas/houce/Alphaearth_embedding/GEE_extracted/{save_name}/metadata/{save_name}_download_GEE', exist_ok=True)
            # Save each batch to a separate file
            with open(f'/nas/houce/Alphaearth_embedding/GEE_extracted/{save_name}/metadata/{save_name}_download_GEE/{save_name}_regions_batch_{batch_idx+1}.txt', 'w') as f:
                f.write(batch_js)

            print(f"Batch {batch_idx+1} with {len(batch)} regions saved to {save_name}_regions_batch_{batch_idx+1}.txt")
            print(f"Generated {len(regions)} regions for Earth Engine")


    elif mode=="python":
        js_regions = "var regions = [\n"
        for region in regions:
            js_regions += f"  {{\n"
            js_regions += f"    'name': {region['name']}',\n"
            js_regions += f"    'geometry': {region['geometry']}\n"
            js_regions += f"    'time_start': {region['time_start']}\n"
            js_regions += f"    'time_end': {region['time_end']}\n"
            js_regions += f"  }},\n"
        js_regions += "]"


        # Split regions into groups of 25
        batched_regions = []
        for i in range(0, len(regions), batch_size):
            batch = regions[i:i+batch_size]
            batched_regions.append(batch)

            # os.makedirs(f'/code/GEE_download/{save_name}/{save_name}_download_python/', exist_ok=True)

            json_batch = [
                {
                    "name": region["name"],
                    "geometry": region["geometry"],
                    "time_start": region["time_start"],
                    "time_end": region["time_end"]
                }
                for region in batch
            ]
            os.makedirs(f"/nas/houce/Alphaearth_embedding/GEE_extracted/{save_name}/metadata/{save_name}_download_python/", exist_ok=True)
            with open(f'/nas/houce/Alphaearth_embedding/GEE_extracted/{save_name}/metadata/{save_name}_download_python/{save_name}_regions_batch_{i//batch_size+1}.json', 'w', encoding='utf-8') as f_json:
                json.dump(json_batch, f_json, ensure_ascii=False, indent=4)
            print(f"Batch {i//batch_size+1} with {len(batch)} regions saved to {save_name}_regions_batch_{i//batch_size+1}.json")

# save_name = f"Africa_grid_{bound[0]}_{bound[1]}_{bound[2]}_{bound[3]}"
generate_code_script(50, save_name, grid_dict, mode='python')

Batch 1 with 50 regions saved to All_exist_slum_regions_batch_1.json
Batch 2 with 50 regions saved to All_exist_slum_regions_batch_2.json
Batch 3 with 50 regions saved to All_exist_slum_regions_batch_3.json
Batch 4 with 50 regions saved to All_exist_slum_regions_batch_4.json
Batch 5 with 50 regions saved to All_exist_slum_regions_batch_5.json
Batch 6 with 50 regions saved to All_exist_slum_regions_batch_6.json
Batch 7 with 50 regions saved to All_exist_slum_regions_batch_7.json
Batch 8 with 50 regions saved to All_exist_slum_regions_batch_8.json
Batch 9 with 50 regions saved to All_exist_slum_regions_batch_9.json
Batch 10 with 50 regions saved to All_exist_slum_regions_batch_10.json
Batch 11 with 50 regions saved to All_exist_slum_regions_batch_11.json
Batch 12 with 50 regions saved to All_exist_slum_regions_batch_12.json
Batch 13 with 50 regions saved to All_exist_slum_regions_batch_13.json
Batch 14 with 50 regions saved to All_exist_slum_regions_batch_14.json
Batch 15 with 50 regions